# Input Data and Packages

In [1]:
###############
# Environment 
###############
import networkx as nx
import itertools
import pandas as pd
import time
import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [2]:
################
# Data Loading 
################
network = pd.read_csv('./Experimental_Network.csv')
print("Network Shape:", network.shape)
network.tail()

Network Shape: (240540, 2)


,ENSG1,ENSG2
240535,ENSG00000144635,ENSG00000269335
240536,ENSG00000063245,ENSG00000129354
240537,ENSG00000269335,ENSG00000124151
240538,ENSG00000033867,ENSG00000006611
240539,ENSG00000124151,ENSG00000133422


In [3]:
###################
# Imput Big Network 
###################
G = nx.read_edgelist('./Experimental_Network.csv' ,
                     create_using = nx.MultiGraph(), 
                     nodetype = str,  delimiter=',')

In [4]:
###############
# Input Cancer Gene Table 
###############
cancer = pd.read_csv('./Cancer_Table.csv')
print("Cancer Table Shape:", cancer.shape)

Cancer Table Shape: (723, 21)


In [5]:
######################################
# Discard non-cancer genes from the Big Network, leaving only cancer genes left.
######################################
full_nodes = set(G.nodes)
cancer_nodes = set(cancer['Synonyms'].tolist())
discard_nodes = full_nodes.difference(cancer_nodes)
G.remove_nodes_from(discard_nodes)

In [6]:
############################
# Re-add a new full big network
############################
G_initial = nx.read_edgelist('./Experimental_Network.csv' ,
                     create_using = nx.MultiGraph(), 
                     nodetype = str,  delimiter = ',')

# Find L1 neighbors

In [7]:
############################
# From full network, use cancer genes as starting point, Find L1 neighbors of cancer gene
# L1_list has full list of starting point cancer genes with L1 neighbors subnetworks
############################
L1_list = []
for i in range(len(G.nodes)):
    L1_subnetwork = nx.bfs_tree(G_initial, list(G.nodes())[i], depth_limit = 1)
    L1_list.append(L1_subnetwork)

In [8]:
############################
# see if subgraph shared nodes over 1/2
# full_combo是含有shared nodes over 1/2的哪兩個graph的 list
############################
full_combo=[]
for i in range(701):
    for j in range(701):
        if len(set(L1_list[i].nodes).intersection(set(L1_list[j].nodes)))/len(set(L1_list[i].nodes)) >= 0.5:
            combo=(i,j)
            full_combo.append(combo)
        elif len(set(L1_list[i].nodes).intersection(set(L1_list[j].nodes)))/len(set(L1_list[j].nodes)) >= 0.5:
            combo=(i,j)
            full_combo.append(combo)
        else:pass
full_combo

[(0, 0),
 (0, 383),
 (0, 397),
 (0, 574),
 (0, 624),
 (0, 658),
 (0, 662),
 (0, 667),
 (0, 676),
 (1, 1),
 (1, 186),
 (1, 423),
 (1, 442),
 (1, 443),
 (1, 574),
 (1, 633),
 (1, 647),
 (1, 658),
 (1, 662),
 (1, 667),
 (1, 676),
 (1, 678),
 (1, 682),
 (1, 689),
 (2, 2),
 (2, 31),
 (2, 658),
 (2, 662),
 (3, 3),
 (3, 658),
 (3, 662),
 (4, 4),
 (4, 201),
 (4, 297),
 (4, 462),
 (4, 477),
 (4, 553),
 (4, 574),
 (4, 601),
 (4, 607),
 (4, 636),
 (4, 651),
 (4, 658),
 (4, 661),
 (4, 662),
 (4, 676),
 (4, 681),
 (4, 688),
 (4, 690),
 (4, 694),
 (4, 699),
 (5, 5),
 (5, 653),
 (5, 658),
 (5, 662),
 (5, 667),
 (5, 676),
 (5, 682),
 (6, 6),
 (6, 23),
 (6, 42),
 (6, 69),
 (6, 73),
 (6, 93),
 (6, 120),
 (6, 123),
 (6, 137),
 (6, 141),
 (6, 185),
 (6, 188),
 (6, 195),
 (6, 209),
 (6, 249),
 (6, 261),
 (6, 262),
 (6, 268),
 (6, 278),
 (6, 293),
 (6, 294),
 (6, 295),
 (6, 309),
 (6, 320),
 (6, 345),
 (6, 392),
 (6, 442),
 (6, 443),
 (6, 463),
 (6, 468),
 (6, 491),
 (6, 494),
 (6, 531),
 (6, 553),
 (6, 561

# Merge Network

In [9]:
############################
# 先merge L1_list裡的第一組合，之後再把後面的組合一一與一直變大的第一組合merge起來
# full_merge is the final merged big network
############################
merge1 = nx.compose(L1_list[0],L1_list[383])

for z in range(len(full_combo)):
    full_merge = nx.compose(merge1, L1_list[full_combo[z][1]])

# Find Motif

In [63]:
############################
# 先做出一個target三角形，用在之後從full_merge網路尋找三角形的對象
############################
target = nx.Graph()
target.add_edges_from([(1,2),(1,3),(2,3)])

In [64]:
############################
# 找出full_merge網路裡是三角形形狀的組合量
############################
n = 0
nodes = full_merge.nodes()
full_merge = nx.DiGraph.to_undirected(full_merge)
for sub_nodes in itertools.product(*[nodes, nodes, nodes]):
    subg = full_merge.subgraph(sub_nodes)
    mot_match = nx.is_isomorphic(subg, target)
    if mot_match == True:
        n += 1
        print(n)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270


In [65]:
############################
# 找出是三角形形狀並至少含有一個cancer gene的組合量
############################
n = 0
nodes = full_merge.nodes()
full_merge = nx.DiGraph.to_undirected(full_merge)
canc_node = G.nodes
for sub_nodes in itertools.product(*[canc_node, nodes, nodes]):
    subg = full_merge.subgraph(sub_nodes)
    mot_match = nx.is_isomorphic(subg, target)
    if mot_match == True:
        n += 1
        print(n)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216


In [66]:
############################
# 找出是三角形形狀並至少含有兩個cancer gene的組合量
############################
n = 0
nodes = full_merge.nodes()
full_merge = nx.DiGraph.to_undirected(full_merge)
canc_node = G.nodes
for sub_nodes in itertools.product(*[canc_node, canc_node, nodes]):
    subg = full_merge.subgraph(sub_nodes)
    mot_match = nx.is_isomorphic(subg, target)
    if mot_match == True:
        n += 1
        print(n)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162


In [67]:
############################
# 找出是三角形形狀並含有三個cancer gene的組合量
############################
n = 0
nodes = full_merge.nodes()
full_merge = nx.DiGraph.to_undirected(full_merge)
canc_node = G.nodes
for sub_nodes in itertools.product(*[canc_node, canc_node, canc_node]):
    subg = full_merge.subgraph(sub_nodes)
    mot_match = nx.is_isomorphic(subg, target)
    if mot_match == True:
        n += 1
        print(n)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108


In [68]:
True_frequency1 = 216/270
True_frequency2 = 162/270
True_frequency3 = 108/270